# Experiment analysis: Size Varying sprite discs
## animals are tectum MTZ ablated SAGFF(LF)81C with UAS:NTR-mcherry and controls (no mcherry expression visible)

In [1]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2


import sys
import os
os.chdir(r'C:\\Users\\johannes\\Dropbox\\python\\zFishBehavior\\dishGroupBehavior')
import numpy as np
import functions.matrixUtilities_joh as mu
import math
import matplotlib.pyplot as plt
import tkFileDialog
import models.experiment as xp
import models.experiment_set as es

import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import scipy.stats as stats
import glob
from datetime import datetime

import functions.paperFigureProps as pfp
pfp.paper()

Populating the interactive namespace from numpy and matplotlib
using plot settings for paper plots


Pre-Analyze all experiments only if necessary, this takes a couple of minutes! Experiment summary csv files are saved to disk.

In [ ]:
startDir='E:\\b\\2017\\20171206_TectumMTZ_sizeTune_a\\'
episodeDur=5
aviPath=glob.glob(startDir+'*.avi')
posPath=glob.glob(startDir+'PositionTxt*.txt')
PLPath=glob.glob(startDir+'PL*.txt')
info=pd.DataFrame({'aviPath': aviPath})
info['txtPath']=posPath
info['pairList']=PLPath
info['epiDur']=episodeDur
info['episodes']=-1
info['set']=0
info['anID']=0
info['inDish']=10

csvFile=startDir+'csvProcess_fileList.csv'
info.to_csv(csvFile,encoding='utf-8')
csvFile

In [ ]:
birthIndex=[0,0,0,0,0,
           0,0,0,0,0,
           0,0,0,0,0]

birthDay=['2017-11-15']

treatment=np.array([0,1,2,3,0,1,2,3,0,1,2,3,0,1,2])


birthTime='09-00'

birthDay_all=[datetime.strptime(birthDay[x]+'-'+birthTime,'%Y-%m-%d-%H-%M') for x in birthIndex]

In [ ]:


def readExperiment(keepData=False):
    tmp=es.experiment_set(csvFile=csvFile,episodes=-1,recomputeAnimalSize=False,birthDay=birthDay_all)
    if keepData:
        return tmp
    else:
        return 1

expSet=readExperiment(keepData=True)
print 'done reading experiment'


In [ ]:
summaryFile=posPath[0][:-4]+'_siSummary_epi'+str(episodeDur)+'.csv'

df=pd.read_csv(summaryFile,index_col=0,sep=',')
df['episode']=[x.strip().replace('_','') for x in df['episode']]
#df=df[df['animalIndex']!=14]
#df=df[df['animalIndex'].isin(np.arange(0,14,2))]
df

In [ ]:
d=df.time
r=datetime(2017,1,1)
t2=[pd.to_datetime(x).replace(day=1,month=1)for x in df.time]

t3=[(x-r)/pd.Timedelta('1 hour') for x in t2]
df['t2']=t2
df['t3']=t3
df

## Habituation or Fatigue within 20 hours?

Plot shoaling index during closed loop skype episodes over time.

In [ ]:
sns.tsplot(data=df, time="t3",value="si",unit="animalIndex",condition="episode",estimator=np.nanmean,interpolate=False,err_style="ci_bars");
plt.xlim([0,24])

In [ ]:
idx=(df['inDishTime']<350) & (df['inDishTime']>45)
episodeNames=df['episode'].unique()
dfDR=df[idx]
tmp=dfDR.groupby(['episode','animalIndex'],sort=True)['si']
xax=np.array([x[-2:] for x in episodeNames]).astype('int')/1000.0
xax=xax*25*10
xax.sort()

err=tmp.std().unstack().values.T

fig, axes = plt.subplots(nrows=2, ncols=1, sharex='col', sharey=False,
                               gridspec_kw={'height_ratios': [2, 1]},
                               figsize=(10, 7))
col=treatment


lab=np.array(["a", "b",'c','d','e'])[col[:5]]

co=sns.color_palette("Dark2", 5)

xt=xax.copy()
xt[0]=0
axes[0]=tmp.mean().unstack().plot(x=xax,kind='line',marker='o',yerr=err,
                                  linestyle=':',ax=axes[0],color=[co[c] for c in col],legend=False,
                                 xticks=xt)
dfSkype=df[idx]
dfSkype=dfSkype[dfSkype['episode']==episodeNames[-1]]
mn=dfSkype.si.mean()
er=dfSkype.si.std()


for s in xax:
    c=plt.Circle((s,0),s/10,color='k')
    axes[1].add_artist(c)
    
axes[1].set_aspect('equal')
axes[1].set_xlabel('disc diameter [mm]')
axes[0].set_ylabel('attraction index')
plt.xlim([0,xax.max()+xax.max()*0.1])
plt.setp(axes[1].get_yticklabels(), visible=False)
axes[1].set_ylim([-11,11]);
lines, labels = axes[0].get_legend_handles_labels()
axes[0].legend([lines[x] for x in[0,1,2,3,4]], [labels[x] for x in[0,1,2,3,4]], labels=lab)
axes[0].axhline(0,ls=':',color='k')
#axes[0].xaxis.set_major_formatter(FormatStrFormatter('%0.1f'))
axes[0].set_title('Individual Pair Disc Size Tuning, Night vs. Day, n=8 pairs');


In [ ]:
df['treatment']=treatment[df.animalIndex.values]
df.head()

In [ ]:
social=df[idx].groupby(['treatment','episode','animalIndex']).si.mean().reset_index()
treatName=np.array(['t+d+','t+d-','t-d+','t-d-'])
social['treatName']=treatName[social.treatment.values.astype('int')]
ax=sns.swarmplot(data=social,x='episode',hue='treatName',y='si',zorder=1,linewidth=1,edgecolor='gray')
#sns.swarmplot(data=dfSIs,x='condition',y='si',zorder=1,linewidth=1,edgecolor='gray',color='white')


sns.pointplot(x="episode", y="si", hue='treatName',data=social,ci=None,zorder=100,scale=2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:4], labels[:4])
plt.xlabel('dot size [mm]')
plt.ylabel('attraction')
plt.xticks(range(7),['0','0.5','1','2','4','8','16'])
plt.axhline(0,ls=':',color='gray')
sns.despine()

In [ ]:
social=df[idx].groupby(['treatment','episode','animalIndex']).avgSpeed.mean().reset_index()
social['treatName']=treatName[social.treatment.values.astype('int')]
ax=sns.swarmplot(data=social,x='episode',hue='treatName',y='avgSpeed',zorder=1,linewidth=1,edgecolor='gray')
#sns.swarmplot(data=dfSIs,x='condition',y='si',zorder=1,linewidth=1,edgecolor='gray',color='white')


sns.pointplot(x="episode", y="avgSpeed", hue='treatName',data=social,ci=None,zorder=100,scale=2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:4], labels[:4])
plt.xlabel('dot size [mm]')
plt.ylabel('average Speed [mm/sec]')
plt.xticks(range(7),['0','0.5','1','2','4','8','16'])
plt.axhline(0,ls=':',color='gray')
sns.despine()

In [ ]:
social=df[idx].groupby(['treatment','episode','animalIndex']).thigmoIndex.mean().reset_index()
social['treatName']=treatName[social.treatment.values.astype('int')]
ax=sns.swarmplot(data=social,x='episode',hue='treatName',y='thigmoIndex',zorder=1,linewidth=1,edgecolor='gray')
#sns.swarmplot(data=dfSIs,x='condition',y='si',zorder=1,linewidth=1,edgecolor='gray',color='white')


sns.pointplot(x="episode", y="thigmoIndex", hue='treatName',data=social,ci=None,zorder=100,scale=2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:4], labels[:4])
plt.xlabel('dot size [mm]')
plt.ylabel('thigmotaxis')
plt.xticks(range(7),['0','0.5','1','2','4','8','16'])
plt.axhline(0,ls=':',color='gray')
sns.despine()

In [ ]:
nmatAll=np.array([y.pair.animals[0].ts.neighborMat() for y in expSet.ee])
nmatAllC=np.array([y.sPair[0].animals[0].ts.neighborMat() for y in expSet.ee])

In [ ]:
levels=df['episode'].unique()
ans=df['animalIndex'].unique()
avg=np.zeros((len(ans),len(levels),nmatAll.shape[1],nmatAll.shape[2]))
avgC=np.zeros((len(ans),len(levels),nmatAll.shape[1],nmatAll.shape[2]))

In [ ]:
print levels
print ans

In [ ]:
for an in ans:
    for i in range(len(levels)):
        ix=np.where((df['episode']==levels[i]) & (df['animalIndex']==an) & idx)[0]
        avg[an,i,:,:]=nmatAll[ix,:,:].mean(axis=0)
        avgC[an,i,:,:]=nmatAllC[ix,:,:].mean(axis=0)
        

In [ ]:
fig, axes = plt.subplots(nrows=15, ncols=7, sharex='col', sharey=True,figsize=(10, 30))
m=np.nanpercentile(avg,95)
trLab=treatName
for an in ans:
    for i in range(len(levels)):
        axes[an,i].imshow(avg[an,i,:,:],clim=[0,m],extent=[-31,31,-31,31])
        axes[an,i].set_title('a:'+str(an)+trLab[treatment[an]]+ 's:'+levels[i][-2:],fontsize=10)

In [ ]:
levels=df['episode'].unique()
treat=[0,1,2,3]
avgT=np.zeros((len(treat),len(levels),nmatAll.shape[1],nmatAll.shape[2]))
avgTC=np.zeros((len(treat),len(levels),nmatAll.shape[1],nmatAll.shape[2]))

In [ ]:
for an in treat:
    for i in range(len(levels)):
        ix=np.where((df['episode']==levels[i]) & (df['treatment']==an) & idx)[0]
        avgT[an,i,:,:]=nmatAll[ix,:,:].mean(axis=0)
        avgTC[an,i,:,:]=nmatAllC[ix,:,:].mean(axis=0)
        

In [ ]:
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower ofset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax/(vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highets point in the colormap's range.
          Defaults to 1.0 (no upper ofset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap

In [ ]:
import matplotlib.gridspec as gridspec

pfp.paper()
inToCm=2.54

outer = gridspec.GridSpec(2, 2, width_ratios = [5,.1], wspace = 0.05) 
#make nested gridspecs
gs2 = gridspec.GridSpecFromSubplotSpec(4, 7, subplot_spec = outer[0])
gs3 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[1])
#gs4 = gridspec.GridSpecFromSubplotSpec(2, 7, subplot_spec = outer[2])

fig = plt.figure(figsize=(11/inToCm,11/inToCm))
axes = [fig.add_subplot(gs2[i]) for i in range(28)]
axesCB=[fig.add_subplot(gs3[i]) for i in range(1)]

axesSP=fig.add_subplot(outer[2])

m=np.nanpercentile(avgT,99)
orig_cmap = matplotlib.cm.bwr
cmap=shiftedColorMap(orig_cmap,midpoint=1-(m/(m+1)))
trLab=treatName
pal=['gray','r','g','m']
for an in treat:
    for i in range(len(levels)):
        ind=i+(7*an)
        im = axes[ind].imshow(avgT[an,i,:,:],clim=[0,m],extent=[-31,31,-31,31],origin='lower',cmap=cmap)
        axes[ind].tick_params(axis='y', which='both',length=0)
        axes[ind].tick_params(axis='x', which='both',length=0)
        axes[ind].set_xticks([])
        axes[ind].set_yticks([])
        axes[ind].spines['top'].set_color('white')
        axes[ind].spines['bottom'].set_color('white')
        axes[ind].spines['left'].set_color('white')
        axes[ind].spines['right'].set_color('white')
        #axes[an,i].set_title('a:'+str(an)+trLab[an]+ 's:'+levels[i][-2:],fontsize=10)
        #if an==1:
            #axes[ind].set_xlabel(int(levels[i][-2:])/2.,fontsize=10)
        if i==0:
            axes[ind].set_title(trLab[an],fontsize=8,color=pal[an])
            
        if (i==5)&(an==0):
            axes[ind].set_title('neighbor density',fontsize=9)

#plt.colorbar(im)
cbar=plt.colorbar(im,cax=axesCB[0],ticks=np.round([0,1,m-0.1]))

plt.subplots_adjust(wspace=0, hspace=0.1)

social=df[idx].groupby(['treatment','episode','animalIndex']).si.mean().reset_index()
social['xpretty']=[int(ss[-2:])/2. for ss in social.episode]
#treatName=np.array(['CTR+MTZ','C81+MTZ'])
social['treatName']=treatName[social.treatment.values.astype('int')]
sns.swarmplot(data=social,x='xpretty',hue='treatName',y='si',
              zorder=1,linewidth=1,edgecolor='gray',ax=axesSP,palette=pal,alpha=0.7)
#sns.swarmplot(data=dfSIs,x='condition',y='si',zorder=1,linewidth=1,edgecolor='gray',color='white')


sns.pointplot(x="xpretty", y="si", hue='treatName',data=social,ci=None,zorder=100,scale=2,ax=axesSP,palette=pal,
              linewidth=1,edgecolor='gray')
axesSP.spines['top'].set_color('white')
axesSP.spines['bottom'].set_color('white')
#axesSP.spines['left'].set_color('white')
axesSP.spines['right'].set_color('white')
axesSP.tick_params(axis='x', which='both',length=0)
#axes[ind].set_xticks([])
#axesSP.set_xticks([])
axesSP.yaxis.tick_left()
axesSP.set_xlabel('dot diameter [mm]')
axesSP.set_ylabel('attraction')
handles, labels = axesSP.get_legend_handles_labels()
axesSP.legend(handles[:4], labels[:4])
#plt.xlabel('dot size [mm]')
#plt.ylabel('attraction')
#plt.xticks(range(7),['0','0.5','1','2','4','8','16'])
axesSP.axhline(0,ls=':',color='k')
#sns.despine()

#fig.text(0.5, -0.11, 'dot diameter [mm]', ha='center',fontsize=12)

#figPath='C:\\Users\\johannes\\Dropbox\\00_CurrentManuscript\\5_MTZ_mapsAndAttraction.svg'
#plt.savefig(figPath)

In [ ]:
trLab

In [ ]:
treatName

In [ ]:
pfp.paper()
fig, ax = plt.subplots(figsize=(2/inToCm,4.5/inToCm))

social=df[idx].groupby(['treatment','animalIndex']).avgSpeed.mean().reset_index()
treatName=np.array(['CTR+MTZ','C81+MTZ'])
social['treatName']=treatName[social.treatment.values.astype('int')]
sns.boxplot(y="avgSpeed", x='treatName',data=social,ax=ax,palette=pal,linewidth=2)

#sns.swarmplot(data=dfSIs,x='condition',y='si',zorder=1,linewidth=1,edgecolor='gray',color='white')

# Select which box you want to change    
for i,artist in enumerate(ax.artists):
# Change the appearance of that box
    #mybox.set_linecolor('red')
    artist.set_edgecolor('k')
    for j in range(i*6,i*6+6):
        line = ax.lines[j]
        line.set_color('k')
        line.set_mfc('k')
        line.set_mec('k')

sns.swarmplot(data=social,x='treatName',y='avgSpeed',zorder=100,linewidth=1,ax=ax,palette=pal,alpha=0.7,edgecolor='k')

plt.xlabel('')
plt.xticks([])
plt.ylabel('average Speed \n [mm/sec]')
#plt.xticks(range(7),['0','0.5','1','2','4','8','16'])
#plt.axhline(0,ls=':',color='gray')
#for tick in ax.xaxis.get_major_ticks():
#    tick.label.set_rotation(45)
plt.ylim([0,7])
sns.despine()
plt.subplots_adjust(wspace=0, hspace=0)
#figPath='C:\\Users\\johannes\\Dropbox\\00_CurrentManuscript\\5_MTZ_AvgSpeed.svg'
#plt.savefig(figPath)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=7, sharex=True, sharey=True)
m=np.nanpercentile(avg,95)
trLab=['c','abl']
for an in treat:
    for i in range(len(levels)):
        profile=avgT[an,i,:,29:31].mean(axis=1)
        axes[an,i].plot(profile,np.arange(profile.shape[0])-30)
        axes[an,i].set_title('a:'+str(an)+trLab[an]+ 's:'+levels[i][-2:],fontsize=10)
        axes[an,i].axhline(0,ls=':',color='gray')
        profile=avgTC[an,i,:,29:31].mean(axis=1)
        axes[an,i].plot(profile,np.arange(profile.shape[0])-30,ls=':',color='k')
        axes[an,i].axvline(0,color='k',linewidth=1)
        axes[an,i].set_ylim([-30,30])

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5, sharex='col', sharey=True,figsize=(12, 5))
m=np.nanpercentile(avg,95)
trLab=['c','abl']
for an in treat:
    for i in range(len(levels)):
        axes[an,i].plot(avgT[an,i,29:31,:].mean(axis=0))
        axes[an,i].set_title('a:'+str(an)+trLab[an]+ 's:'+levels[i][-2:],fontsize=10)
        axes[an,i].axvline(30)